# Representing text

If we want to solve Natural Language Processing (NLP) tasks with neural networks, we need some way to represent text as tensors. Computers already represent textual characters as numbers that map to fonts on your screen using encodings such as ASCII or UTF-8.

<img alt="Image showing diagram mapping a character to an ASCII and binary representation" src="images/2-represent-text-as-tensor-checkpoint-1.png" align="middle" />

We understand what each letter **represents**, and how all characters come together to form the words of a sentence. However, computers by themselves do not have such an understanding, and a neural network has to learn the meaning during training.

Therefore, we can use different approaches when representing text:
* **Character-level representation**, when we represent text by treating each character as a number. Given that we have $C$ different characters in our text corpus, the word *Hello* would be represented by $5\times C$ tensor. Each letter would correspond to a tensor column in one-hot encoding.
* **Word-level representation**, when we create a **vocabulary** of all words in our text sequence or sentence(s), and then represent each word using one-hot encoding. This approach is somehow better, because each letter by itself does not have much meaning, and thus by using higher-level semantic concepts - words - we simplify the task for the neural network. However, given a large dictionary size, we need to deal with high-dimensional sparse tensors.  For example, if we have a vocabulary size of 10,000 different words.  Then each word would have an one-hot encoding length of 10,000; hence the high-dimensional.

To unify those approaches, we typically call an atomic piece of text **a token**. In some cases tokens can be letters, in other cases - words, or parts of words.

> For example, we can choose to tokenize *indivisible* as `in`-`divis`-`ible`, where the `#` sign represents that the token is a continuation of the previous word. This would allow the root `divis` to always be represented by one token, corresponding to one core meaning.

The process of converting text into a sequence of tokens is called **tokenization**. Next, we need to assign each token to a number, which we can feed into a neural network. This is called **vectorization**, and is normally done by building a token vocabulary.  

Let's start by installing some required Python packages we'll use in this module.

In [4]:
import torch
import torchtext
import os
import collections


In [7]:
# Ensure the 'data' directory exists
os.makedirs('./data', exist_ok=True)

# Load AG_NEWS dataset
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

# Create an iterator for the train_dataset
train_iterator = iter(train_dataset)

# Example: Print the first item
print(next(train_iterator))

(3, "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")


In [9]:
# Load AG_NEWS dataset
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

# Iterate over the first 5 samples in the train_dataset
for i, (label, text) in zip(range(5), train_dataset):
    print(f"**{classes[label - 1]}** -> {text}\n")

**Business** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.

**Business** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.

**Business** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.

**Business** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.

**Business** -> Oil prices soa

In [10]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

In [11]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

In [12]:
first_sentence = train_dataset[0][1]
second_sentence = train_dataset[1][1]

f_tokens = tokenizer(first_sentence)
s_tokens = tokenizer(second_sentence)

print(f'\nfirst token list:\n{f_tokens}')
print(f'\nsecond token list:\n{s_tokens}')


first token list:
['wall', 'st', '.', 'bears', 'claw', 'back', 'into', 'the', 'black', '(', 'reuters', ')', 'reuters', '-', 'short-sellers', ',', 'wall', 'street', "'", 's', 'dwindling\\band', 'of', 'ultra-cynics', ',', 'are', 'seeing', 'green', 'again', '.']

second token list:
['carlyle', 'looks', 'toward', 'commercial', 'aerospace', '(', 'reuters', ')', 'reuters', '-', 'private', 'investment', 'firm', 'carlyle', 'group', ',', '\\which', 'has', 'a', 'reputation', 'for', 'making', 'well-timed', 'and', 'occasionally\\controversial', 'plays', 'in', 'the', 'defense', 'industry', ',', 'has', 'quietly', 'placed\\its', 'bets', 'on', 'another', 'part', 'of', 'the', 'market', '.']


In [ ]:
# Assuming you have defined your tokenizer function
def tokenizer(line):
    # Your tokenizer logic here
    pass

# Count tokens in the dataset
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))

# Create a vocabulary with a minimum frequency of 1
vocab = torchtext.vocab.Vocab(counter)

# Load GloVe embeddings (adjust the vectors parameter accordingly)
glove_vectors = torchtext.vocab.GloVe(name='6B', dim=100)

# Associate GloVe vectors with the vocabulary
vocab.set_vectors(glove_vectors.stoi, glove_vectors.vectors, glove_vectors.dim)


.vector_cache/glove.6B.zip:   1%|▋                                                                                                       | 5.88M/862M [02:29<11:01:41, 21.6kB/s]

In [ ]:
word_lookup = [list((vocab[w], w)) for w in f_tokens]
print(f'\nIndex lockup in 1st sentence:\n{word_lookup}')

word_lookup = [list((vocab[w], w)) for w in s_tokens]
print(f'\nIndex lockup in 2nd sentence:\n{word_lookup}')

In [ ]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

def encode(x):
    return [vocab.stoi[s] for s in tokenizer(x)]

vec = encode(first_sentence)
print(vec)

In [ ]:
def decode(x):
    return [vocab.itos[i] for i in x]

decode(vec)

In [ ]:
from torchtext.data.utils import ngrams_iterator

bi_counter = collections.Counter()
for (label, line) in train_dataset:
    bi_counter.update(ngrams_iterator(tokenizer(line),ngrams=2))
bi_vocab = torchtext.vocab.Vocab(bi_counter, min_freq=2)

print(f"Bigram vocab size = {len(bi_vocab)}")

In [ ]:
def encode(x):
    return [bi_vocab.stoi[s] for s in tokenizer(x)]

encode(first_sentence)